In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python2'
from pyspark import SparkContext
sc = SparkContext('local[4]')

In [24]:
from pyspark import SQLContext, Row
sqlCtx = SQLContext(sc)
myKeyValue = [(1,'Alice'),(2,'Bob'),(3,'Charlie'),(3,'Chuck'),(1,'Alice')]
rdd = sc.parallelize(myKeyValue)#.map(lambda x: Row(x))

In [30]:
from pyspark.sql.types import *
fields = [StructField('Key',IntegerType(),True),StructField('Value',StringType(),True)]
schema = StructType(fields)
df = sqlCtx.createDataFrame(rdd, schema)

In [31]:
df.show()

+---+-------+
|Key|  Value|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
|  3|  Chuck|
|  1|  Alice|
+---+-------+



In [32]:
df.explain(extended=True)

== Parsed Logical Plan ==
LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Analyzed Logical Plan ==
Key: int, Value: string
LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Optimized Logical Plan ==
LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Physical Plan ==
Scan PhysicalRDD[Key#3,Value#4]

Code Generation: true


In [33]:
df_distinct = df.distinct()
df_distinct.explain(extended=True)

== Parsed Logical Plan ==
Aggregate [Key#3,Value#4], [Key#3,Value#4]
 LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Analyzed Logical Plan ==
Key: int, Value: string
Aggregate [Key#3,Value#4], [Key#3,Value#4]
 LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Optimized Logical Plan ==
Aggregate [Key#3,Value#4], [Key#3,Value#4]
 LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Physical Plan ==
TungstenAggregate(key=[Key#3,Value#4], functions=[], output=[Key#3,Value#4])
 TungstenExchange hashpartitioning(Key#3,Value#4)
  TungstenAggregate(key=[Key#3,Value#4], functions=[], output=[Key#3,Value#4])
   Scan PhysicalRDD[Key#3,Value#4]

Code Generation: true


In [36]:
df_distinct.show()

+---+-------+
|Key|  Value|
+---+-------+
|  1|  Alice|
|  3|Charlie|
|  2|    Bob|
|  3|  Chuck|
+---+-------+



In [37]:
df_duplicates_dropped = df.dropDuplicates(['Key'])
df_duplicates_dropped.explain(extended=True)

== Parsed Logical Plan ==
Aggregate [Key#3], [Key#3,FIRST(Value#4) AS Value#10]
 LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Analyzed Logical Plan ==
Key: int, Value: string
Aggregate [Key#3], [Key#3,FIRST(Value#4) AS Value#10]
 LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Optimized Logical Plan ==
Aggregate [Key#3], [Key#3,FIRST(Value#4) AS Value#10]
 LogicalRDD [Key#3,Value#4], MapPartitionsRDD[44] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Physical Plan ==
SortBasedAggregate(key=[Key#3], functions=[(first(Value#4),mode=Final,isDistinct=false)], output=[Key#3,Value#10])
 ConvertToSafe
  TungstenSort [Key#3 ASC], false, 0
   TungstenExchange hashpartitioning(Key#3)
    ConvertToUnsafe
     SortBasedAggregate(key=[Key#3], functions=[(first(Value#4),mode=Partial,isDistinct=false)], output=[Key#3,first#13])
      ConvertToSafe

In [38]:
df_duplicates_dropped.show()

+---+-------+
|Key|  Value|
+---+-------+
|  1|  Alice|
|  2|    Bob|
|  3|Charlie|
+---+-------+



In [ ]:
sc.stop()